In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Market Research System v1.0 - Report Generation Examples\n",
    "\n",
    "**Version:** 1.0 (2022)\n",
    "\n",
    "**Focus:** Indian Stock Market Analysis and Reporting\n",
    "\n",
    "This notebook demonstrates comprehensive report generation capabilities for the Indian stock market, including:\n",
    "- Daily market analysis reports\n",
    "- Technical indicator calculations\n",
    "- Sector performance analysis\n",
    "- Economic correlation studies\n",
    "- PDF report generation\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📦 Installation and Setup\n",
    "\n",
    "Install required packages for Indian market data analysis:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install required packages\n",
    "!pip install yfinance pandas numpy matplotlib seaborn plotly\n",
    "!pip install ta-lib talib-binary\n",
    "!pip install reportlab fpdf2\n",
    "!pip install requests beautifulsoup4\n",
    "!pip install scipy scikit-learn\n",
    "!pip install plotly-express kaleido"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Import Libraries and Configuration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Core libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import yfinance as yf\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import plotly.graph_objects as go\n",
    "import plotly.express as px\n",
    "from plotly.subplots import make_subplots\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Date and time\n",
    "from datetime import datetime, timedelta, date\n",
    "import pytz\n",
    "\n",
    "# Technical analysis\n",
    "try:\n",
    "    import talib\n",
    "    TALIB_AVAILABLE = True\n",
    "except ImportError:\n",
    "    TALIB_AVAILABLE = False\n",
    "    print(\"TA-Lib not available, using manual calculations\")\n",
    "\n",
    "# Statistics and ML\n",
    "from scipy import stats\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "from sklearn.decomposition import PCA\n",
    "\n",
    "# Report generation\n",
    "from reportlab.lib.pagesizes import letter, A4\n",
    "from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle\n",
    "from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle\n",
    "from reportlab.lib.units import inch\n",
    "from reportlab.lib import colors\n",
    "\n",
    "# File operations\n",
    "import os\n",
    "import io\n",
    "import base64\n",
    "from io import BytesIO\n",
    "\n",
    "# Set style for plots\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "\n",
    "# Configuration\n",
    "pd.set_option('display.max_columns', None)\n",
    "pd.set_option('display.width', None)\n",
    "pd.set_option('display.max_colwidth', None)\n",
    "\n",
    "print(\"✅ All libraries imported successfully!\")\n",
    "print(f\"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\")\n",
    "print(f\"🌍 Timezone: {datetime.now(pytz.timezone('Asia/Kolkata')).strftime('%Z')}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🇮🇳 Indian Stock Market Configuration\n",
    "\n",
    "Define the Indian stock universe and market indices for analysis:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Indian Stock Market Configuration (2022 Focus)\n",
    "\n",
    "# Major Indian Indices\n",
    "INDIAN_INDICES = {\n",
    "    '^NSEI': 'NIFTY 50',\n",
    "    '^BSESN': 'BSE SENSEX',\n",
    "    '^NSEBANK': 'NIFTY BANK',\n",
    "    '^NSEIT': 'NIFTY IT',\n",
    "    '^NSMIDCP': 'NIFTY MIDCAP 50',\n",
    "    'INDIAVIX.NS': 'INDIA VIX'\n",
    "}\n",
    "\n",
    "# Top 30 Indian Stocks by Market Cap (NSE symbols - 2022)\n",
    "TOP_INDIAN_STOCKS = {\n",
    "    # Technology Sector\n",
    "    'TCS.NS': 'Tata Consultancy Services',\n",
    "    'INFY.NS': 'Infosys',\n",
    "    'WIPRO.NS': 'Wipro',\n",
    "    'HCLTECH.NS': 'HCL Technologies',\n",
    "    'TECHM.NS': 'Tech Mahindra',\n",
    "    \n",
    "    # Banking and Financial Services\n",
    "    'HDFCBANK.NS': 'HDFC Bank',\n",
    "    'ICICIBANK.NS': 'ICICI Bank',\n",
    "    'KOTAKBANK.NS': 'Kotak Mahindra Bank',\n",
    "    'AXISBANK.NS': 'Axis Bank',\n",
    "    'SBIN.NS': 'State Bank of India',\n",
    "    \n",
    "    # Energy and Oil\n",
    "    'RELIANCE.NS': 'Reliance Industries',\n",
    "    'ONGC.NS': 'Oil and Natural Gas Corporation',\n",
    "    'IOC.NS': 'Indian Oil Corporation',\n",
    "    'NTPC.NS': 'NTPC Limited',\n",
    "    \n",
    "    # Consumer Goods\n",
    "    'HINDUNILVR.NS': 'Hindustan Unilever',\n",
    "    'ITC.NS': 'ITC Limited',\n",
    "    'NESTLEIND.NS': 'Nestle India',\n",
    "    'BRITANNIA.NS': 'Britannia Industries',\n",
    "    \n",
    "    # Automobiles\n",
    "    'MARUTI.NS': 'Maruti Suzuki',\n",
    "    'TATAMOTORS.NS': 'Tata Motors',\n",
    "    'M&M.NS': 'Mahindra & Mahindra',\n",
    "    \n",
    "    # Pharmaceuticals\n",
    "    'SUNPHARMA.NS': 'Sun Pharmaceutical',\n",
    "    'DRREDDY.NS': 'Dr. Reddys Laboratories',\n",
    "    'CIPLA.NS': 'Cipla',\n",
    "    \n",
    "    # Metals and Mining\n",
    "    'TATASTEEL.NS': 'Tata Steel',\n",
    "    'HINDALCO.NS': 'Hindalco Industries',\n",
    "    'JINDALSTEL.NS': 'Jindal Steel & Power',\n",
    "    \n",
    "    # Others\n",
    "    'LT.NS': 'Larsen & Toubro',\n",
    "    'BHARTIARTL.NS': 'Bharti Airtel',\n",
    "    'TITAN.NS': 'Titan Company'\n",
    "}\n",
    "\n",
    "# Sector Classification\n",
    "SECTOR_CLASSIFICATION = {\n",
    "    'Technology': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS', 'TECHM.NS'],\n",
    "    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS', 'SBIN.NS'],\n",
    "    'Energy': ['RELIANCE.NS', 'ONGC.NS', 'IOC.NS', 'NTPC.NS'],\n",
    "    'Consumer_Goods': ['HINDUNILVR.NS', 'ITC.NS', 'NESTLEIND.NS', 'BRITANNIA.NS'],\n",
    "    'Automobiles': ['MARUTI.NS', 'TATAMOTORS.NS', 'M&M.NS'],\n",
    "    'Pharmaceuticals': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS'],\n",
    "    'Metals': ['TATASTEEL.NS', 'HINDALCO.NS', 'JINDALSTEL.NS']\n",
    "}\n",
    "\n",
    "# Market timings (IST)\n",
    "MARKET_OPEN_TIME = \"09:15\"\n",
    "MARKET_CLOSE_TIME = \"15:30\"\n",
    "INDIAN_TIMEZONE = pytz.timezone('Asia/Kolkata')\n",
    "\n",
    "print(\"🇮🇳 Indian Market Configuration Loaded:\")\n",
    "print(f\"📊 Indices: {len(INDIAN_INDICES)}\")\n",
    "print(f\"📈 Stocks: {len(TOP_INDIAN_STOCKS)}\")\n",
    "print(f\"🏭 Sectors: {len(SECTOR_CLASSIFICATION)}\")\n",
    "\n",
    "# Display sector breakdown\n",
    "for sector, stocks in SECTOR_CLASSIFICATION.items():\n",
    "    print(f\"   {sector}: {len(stocks)} stocks\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📈 Data Fetching Functions\n",
    "\n",
    "Create functions to fetch Indian market data efficiently:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class IndianMarketDataFetcher:\n",
    "    \"\"\"Comprehensive Indian Market Data Fetcher for NSE/BSE stocks\"\"\"\n",
    "    \n",
    "    def __init__(self):\n",
    "        self.indian_tz = pytz.timezone('Asia/Kolkata')\n",
    "        self.data_cache = {}\n",
    "    \n",
    "    def fetch_stock_data(self, symbol, period='1y', interval='1d'):\n",
    "        \"\"\"Fetch stock data with error handling\"\"\"\n",
    "        try:\n",
    "            stock = yf.Ticker(symbol)\n",
    "            data = stock.history(period=period, interval=interval)\n",
    "            \n",
    "            if data.empty:\n",
    "                print(f\"⚠️ No data found for {symbol}\")\n",
    "                return None\n",
    "            \n",
    "            # Add symbol column\n",
    "            data['Symbol'] = symbol\n",
    "            return data\n",
    "            \n",
    "        except Exception as e:\n",
    "            print(f\"❌ Error fetching {symbol}: {str(e)}\")\n",
    "            return None\n",
    "    \n",
    "    def fetch_multiple_stocks(self, symbols, period='1y'):\n",
    "        \"\"\"Fetch data for multiple stocks\"\"\"\n",
    "        stock_data = {}\n",
    "        failed_stocks = []\n",
    "        \n",
    "        print(f\"📊 Fetching data for {len(symbols)} stocks...\")\n",
    "        \n",
    "        for i, symbol in enumerate(symbols):\n",
    "            print(f\"   Fetching {i+1}/{len(symbols)}: {symbol}\", end=\" \")\n",
    "            \n",
    "            data = self.fetch_stock_data(symbol, period)\n",
    "            if data is not None:\n",
    "                stock_data[symbol] = data\n",
    "                print(\"✅\")\n",
    "            else:\n",
    "                failed_stocks.append(symbol)\n",
    "                print(\"❌\")\n",
    "        \n",
    "        print(f\"📈 Successfully fetched: {len(stock_data)} stocks\")\n",
    "        if failed_stocks:\n",
    "            print(f\"⚠️ Failed to fetch: {failed_stocks}\")\n",
    "        \n",
    "        return stock_data, failed_stocks\n",
    "    \n",
    "    def get_market_indices(self, period='1y'):\n",
    "        \"\"\"Fetch Indian market indices data\"\"\"\n",
    "        indices_data = {}\n",
    "        \n",
    "        print(\"📊 Fetching Indian market indices...\")\n",
    "        for symbol, name in INDIAN_INDICES.items():\n",
    "            print(f\"   Fetching {name} ({symbol})\", end=\" \")\n",
    "            data = self.fetch_stock_data(symbol, period)\n",
    "            if data is not None:\n",
    "                indices_data[symbol] = data\n",
    "                print(\"✅\")\n",
    "            else:\n",
    "                print(\"❌\")\n",
    "        \n",
    "        return indices_data\n",
    "    \n",
    "    def calculate_returns(self, data):\n",
    "        \"\"\"Calculate various return metrics\"\"\"\n",
    "        returns_data = {}\n",
    "        \n",
    "        for symbol, df in data.items():\n",
    "            if df is not None and not df.empty:\n",
    "                # Calculate returns\n",
    "                df['Daily_Return'] = df['Close'].pct_change()\n",
    "                df['Cumulative_Return'] = (1 + df['Daily_Return']).cumprod() - 1\n",
    "                \n",
    "                # Calculate metrics\n",
    "                returns_data[symbol] = {\n",
    "                    'total_return': df['Cumulative_Return'].iloc[-1],\n",
    "                    'volatility': df['Daily_Return'].std() * np.sqrt(252),\n",
    "                    'max_drawdown': self.calculate_max_drawdown(df['Close']),\n",
    "                    'sharpe_ratio': self.calculate_sharpe_ratio(df['Daily_Return']),\n",
    "                    'current_price': df['Close'].iloc[-1],\n",
    "                    'price_change': df['Close'].iloc[-1] - df['Close'].iloc[-2] if len(df) > 1 else 0\n",
    "                }\n",
    "        \n",
    "        return returns_data\n",
    "    \n",
    "    def calculate_max_drawdown(self, prices):\n",
    "        \"\"\"Calculate maximum drawdown\"\"\"\n",
    "        peak = prices.expanding().max()\n",
    "        drawdown = (prices - peak) / peak\n",
    "        return drawdown.min()\n",
    "    \n",
    "    def calculate_sharpe_ratio(self, returns, risk_free_rate=0.06):\n",
    "        \"\"\"Calculate Sharpe ratio (assuming 6% risk-free rate for India)\"\"\"\n",
    "        excess_returns = returns.mean() * 252 - risk_free_rate\n",
    "        volatility = returns.std() * np.sqrt(252)\n",
    "        return excess_returns / volatility if volatility != 0 else 0\n",
    "\n",
    "# Initialize data fetcher\n",
    "fetcher = IndianMarketDataFetcher()\n",
    "print(\"✅ Indian Market Data Fetcher initialized!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔧 Technical Indicators Calculator\n",
    "\n",
    "Implement technical indicators for Indian stocks analysis:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class TechnicalIndicators:\n",
    "    \"\"\"Technical Indicators Calculator for Indian Stock Market\"\"\"\n",
    "    \n",
    "    @staticmethod\n",
    "    def sma(data, window):\n",
    "        \"\"\"Simple Moving Average\"\"\"\n",
    "        return data.rolling(window=window).mean()\n",
    "    \n",
    "    @staticmethod\n",
    "    def ema(data, window):\n",
    "        \"\"\"Exponential Moving Average\"\"\"\n",
    "        return data.ewm(span=window).mean()\n",
    "    \n",
    "    @staticmethod\n",
    "    def rsi(data, window=14):\n",
    "        \"\"\"Relative Strength Index\"\"\"\n",
    "        delta = data.diff()\n",
    "        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()\n",
    "        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()\n",
    "        rs = gain / loss\n",
    "        rsi = 100 - (100 / (1 + rs))\n",
    "        return rsi\n",
    "    \n",
    "    @staticmethod\n",
    "    def macd(data, fast=12, slow=26, signal=9):\n",
    "        \"\"\"MACD Indicator\"\"\"\n",
    "        ema_fast = data.ewm(span=fast).mean()\n",
    "        ema_slow = data.ewm(span=slow).mean()\n",
    "        macd_line = ema_fast - ema_slow\n",
    "        signal_line = macd_line.ewm(span=signal).mean()\n",
    "        histogram = macd_line - signal_line\n",
    "        \n",
    "        return macd_line, signal_line, histogram\n",
    "    \n",
    "    @staticmethod\n",
    "    def bollinger_bands(data, window=20, num_std=2):\n",
    "        \"\"\"Bollinger Bands\"\"\"\n",
    "        sma = data.rolling(window=window).mean()\n",
    "        std = data.rolling(window=window).std()\n",
    "        upper_band = sma + (std * num_std)\n",
    "        lower_band = sma - (std * num_std)\n",
    "        \n",
    "        return upper_band, sma, lower_band\n",
    "    \n",
    "    @staticmethod\n",
    "    def stochastic(high, low, close, k_window=14, d_window=3):\n",
    "        \"\"\"Stochastic Oscillator\"\"\"\n",
    "        lowest_low = low.rolling(window=k_window).min()\n",
    "        highest_high = high.rolling(window=k_window).max()\n",
    "        k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low))\n",
    "        d_percent = k_percent.rolling(window=d_window).mean()\n",
    "        \n",
    "        return k_percent, d_percent\n",
    "    \n",
    "    @staticmethod\n",
    "    def atr(high, low, close, window=14):\n",
    "        \"\"\"Average True Range\"\"\"\n",
    "        high_low = high - low\n",
    "        high_close = np.abs(high - close.shift())\n",
    "        low_close = np.abs(low - close.shift())\n",
    "        \n",
    "        true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)\n",
    "        atr = true_range.rolling(window=window).mean()\n",
    "        \n",
    "        return atr\n",
    "    \n",
    "    def calculate_all_indicators(self, df):\n",
    "        \"\"\"Calculate all technical indicators for a dataframe\"\"\"\n",
    "        indicators = df.copy()\n",
    "        \n",
    "        # Moving Averages\n",
    "        indicators['SMA_20'] = self.sma(df['Close'], 20)\n",
    "        indicators['SMA_50'] = self.sma(df['Close'], 50)\n",
    "        indicators['EMA_12'] = self.ema(df['Close'], 12)\n",
    "        indicators['EMA_26'] = self.ema(df['Close'], 26)\n",
    "        \n",
    "        # RSI\n",
    "        indicators['RSI'] = self.rsi(df['Close'])\n",
    "        \n",
    "        # MACD\n",
    "        macd, signal, histogram = self.macd(df['Close'])\n",
    "        indicators['MACD'] = macd\n",
    "        indicators['MACD_Signal'] = signal\n",
    "        indicators['MACD_Histogram'] = histogram\n",
    "        \n",
    "        # Bollinger Bands\n",
    "        bb_upper, bb_middle, bb_lower = self.bollinger_bands(df['Close'])\n",
    "        indicators['BB_Upper'] = bb_upper\n",
    "        indicators['BB_Middle'] = bb_middle\n",
    "        indicators['BB_Lower'] = bb_lower\n",
    "        \n",
    "        # Stochastic\n",
    "        stoch_k, stoch_d = self.stochastic(df['High'], df['Low'], df['Close'])\n",
    "        indicators['Stoch_K'] = stoch_k\n",
    "        indicators['Stoch_D'] = stoch_d\n",
    "        \n",
    "        # ATR\n",
    "        indicators['ATR'] = self.atr(df['High'], df['Low'], df['Close'])\n",
    "        \n",
    "        # Volume indicators\n",
    "        indicators['Volume_SMA'] = self.sma(df['Volume'], 20)\n",
    "        \n",
    "        return indicators\n",
    "\n",
    "# Initialize technical indicators calculator\n",
    "tech_indicators = TechnicalIndicators()\n",
    "print(\"✅ Technical Indicators Calculator initialized!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Fetch Current Market Data\n",
    "\n",
    "Let's fetch the latest Indian market data for analysis:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Fetch Indian market indices data\n",
    "print(\"🇮🇳 Fetching Indian Market Indices Data...\")\n",
    "indices_data = fetcher.get_market_indices(period='6mo')\n",
    "\n",
    "# Fetch top Indian stocks data\n",
    "print(\"\\n📈 Fetching Top Indian Stocks Data...\")\n",
    "stock_symbols = list(TOP_INDIAN_STOCKS.keys())[:15]  # Limit to 15 stocks for demo\n",
    "stocks_data, failed_stocks = fetcher.fetch_multiple_stocks(stock_symbols, period='6mo')\n",
    "\n",
    "# Calculate returns and metrics\n",
    "print(\"\\n📊 Calculating Returns and Metrics...\")\n",
    "stock_metrics = fetcher.calculate_returns(stocks_data)\n",
    "indices_metrics = fetcher.calculate_returns(indices_data)\n",
    "\n",
    "print(f\"\\n✅ Data fetching completed!\")\n",
    "print(f\"📊 Indices loaded: {len(indices_data)}\")\n",
    "print(f\"📈 Stocks loaded: {len(stocks_data)}\")\n",
    "\n",
    "# Display current market status\n",
    "if indices_data and '^NSEI' in indices_data:\n",
    "    nifty_data = indices_data['^NSEI']\n",
    "    current_nifty = nifty_data['Close'].iloc[-1]\n",
    "    nifty_change = current_nifty - nifty_data['Close'].iloc[-2]\n",
    "    nifty_change_pct = (nifty_change / nifty_data['Close'].iloc[-2]) * 100\n",
    "    \n",
    "    print(f\"\\n🇮🇳 NIFTY 50 Status:\")\n",
    "    print(f\"   Current Level: {current_nifty:.2f}\")\n",
    "    print(f\"   Change: {nifty_change:+.2f} ({nifty_change_pct:+.2f}%)\")\n",
    "    \n",
    "    if nifty_change > 0:\n",
    "        print(\"   Market Sentiment: 🟢 Positive\")\n",
    "    else:\n",
    "        print(\"   Market Sentiment: 🔴 Negative\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📈 Market Analysis Functions\n",
    "\n",
    "Create comprehensive analysis functions for Indian market:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class IndianMarketAnalyzer:\n",
    "    \"\"\"Comprehensive Indian Market Analysis Class\"\"\"\n",
    "    \n",
    "    def __init__(self, stocks_data, indices_data):\n",
    "        self.stocks_data = stocks_data\n",
    "        self.indices_data = indices_data\n",
    "        self.analysis_date = datetime.now().strftime('%Y-%m-%d')\n",
    "    \n",
    "    def sector_performance_analysis(self):\n",
    "        \"\"\"Analyze sector-wise performance\"\"\"\n",
    "        sector_performance = {}\n",
    "        \n",
    "        for sector, symbols in SECTOR_CLASSIFICATION.items():\n",
    "            sector_returns = []\n",
    "            sector_data = []\n",
    "            \n",
    "            for symbol in symbols:\n",
    "                if symbol in self.stocks_data:\n",
    "                    data = self.stocks_data[symbol]\n",
    "                    if len(data) > 1:\n",
    "                        # Calculate sector metrics\n",
    "                        daily_return = (data['Close'].iloc[-1] - data['Close'].iloc[-2]) / data['Close'].iloc[-2]\n",
    "                        sector_returns.append(daily_return)\n",
    "                        sector_data.append({\n",
    "                            'symbol': symbol,\n",
    "                            'name': TOP_INDIAN_STOCKS.get(symbol, symbol),\n",
    "                            'price': data['Close'].iloc[-1],\n",
    "                            'change': daily_return * 100,\n",
    "                            'volume': data['Volume'].iloc[-1]\n",
    "                        })\n",
    "            \n",
    "            if sector_returns:\n",
    "                sector_performance[sector] = {\n",
    "                    'avg_return': np.mean(sector_returns) * 100,\n",
    "                    'total_stocks': len(sector_data),\n",
    "                    'gainers': len([r for r in sector_returns if r > 0]),\n",
    "                    'losers': len([r for r in sector_returns if r < 0]),\n",
    "                    'stocks': sector_data\n",
    "                }\n",
    "        \n",
    "        return sector_performance\n",
    "    \n",



    not complted